<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/00_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
#| default_exp colab

# Google Colab utils
> Deal with colab specific format

In [ ]:
#| export
from pathlib import Path
from IPython import get_ipython
from IPython.display import display, Markdown

from fastcore.basics import ifnone

from execnb.nbio import NbCell, read_nb

from nb_helpers.utils import git_main_name, git_origin_repo, git_local_repo, search_cell, git_current_branch

In [ ]:
this_nb = Path("00_colab.ipynb")
notebook = read_nb(this_nb)

In [ ]:
this_nb.resolve()

Path('/Users/tcapelle/wandb/nb_helpers/nbs/00_colab.ipynb')

In [ ]:
git_origin_repo(this_nb.resolve())

'wandb/nb_helpers'

In [ ]:
#| export
def get_colab_url(fname, branch):
    "Get git repo url, to append to colab"
    fname = Path(fname).resolve()
    github_repo = git_origin_repo(fname)
    fname = fname.relative_to(git_local_repo(fname))

    return f"https://colab.research.google.com/github/{github_repo}/blob/{branch}/{str(fname)}"

In [ ]:
get_colab_url(this_nb, branch="master")

'https://colab.research.google.com/github/wandb/nb_helpers/blob/master/nbs/00_colab.ipynb'

In [ ]:
#| exporti
_badge_meta = {"id": "view-in-github", "colab_type": "text"}

In [ ]:
def _create_colab_cell(url, meta={}, tracker=None):
    "Creates a notebook cell with the `Open In Colab` badge"
    tracker = ifnone(tracker, "")
    data = {
        "cell_type": "markdown",
        "metadata": meta,
        "source": f'<a href="{url}" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>\n'
        + tracker,
    }
    return NbCell(-1, data)

In [ ]:
url = get_colab_url(this_nb, branch="master")
cell = _create_colab_cell(url)
display(Markdown(cell["source"]))

<a href="https://colab.research.google.com/github/wandb/nb_helpers/blob/master/nbs/00_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
#| export
def has_colab_badge(nb):
    "Check if notebook has colab badge, returns the cell position, -1 if not present"
    for i, cell in enumerate(nb["cells"]):
        if search_cell(cell, "Open In Colab"):
            return i
    return -1

In [ ]:
assert has_colab_badge(notebook) == 0

In [ ]:
#| export
def create_colab_badge_cell(fname, branch=None, meta={}, tracker=None):
    "Create a colab badge cell from `fname`"
    # get main/master name
    branch = ifnone(branch, git_main_name(fname))
    url = get_colab_url(fname, branch)
    colab_cell = _create_colab_cell(url, meta, tracker)
    return colab_cell

In [ ]:
create_colab_badge_cell(this_nb)

```json
{ 'cell_type': 'markdown',
  'idx_': -1,
  'metadata': {},
  'source': '<a '
            'href="https://colab.research.google.com/github/wandb/nb_helpers/blob/main/nbs/00_colab.ipynb" '
            'target="_parent"><img '
            'src="https://colab.research.google.com/assets/colab-badge.svg" '
            'alt="Open In Colab"/></a>\n'}
```

In [ ]:
#| export
def add_colab_badge(notebook, fname, branch=None, idx=0, meta=_badge_meta, tracker=None):
    "Add a badge to Open In Colab in the `idx` cell"
    idx_colab_badge = has_colab_badge(notebook)
    if idx_colab_badge != -1:
        notebook["cells"].pop(idx_colab_badge)
    colab_cell = create_colab_badge_cell(fname, branch, meta, tracker)
    notebook["cells"].insert(idx, colab_cell)
    return notebook

In [ ]:
nb_with_colab_badge = add_colab_badge(notebook, fname=this_nb)

it should have the colab badge in the first cell now

In [ ]:
assert has_colab_badge(nb_with_colab_badge) == 0

Let's also activate the GPU on colab

In [ ]:
#| exporti
_colab_meta = {
    "accelerator": "GPU",
    "colab": {"include_colab_link": True, "toc_visible": True},
}

In [ ]:
#| export
def add_colab_metadata(notebook, meta=_colab_meta):
    "Adds GPU and colab meta to `notebook`"
    notebook["metadata"].update(_colab_meta)
    return notebook

In [ ]:
add_colab_metadata(notebook)["metadata"]

```json
{ 'accelerator': 'GPU',
  'colab': {'include_colab_link': True, 'toc_visible': True},
  'jupytext': {'split_at_heading': True},
  'kernelspec': { 'display_name': 'Python 3 (ipykernel)',
                  'language': 'python',
                  'name': 'python3'}}
```